In [7]:
import pygame,sys,random
import os
from time import sleep

padWidth = 480
padHeight = 640
rockImage = ['git.png','python.png','keras.png','tensorFlow.png', 'java.png','django.png','spring.png' ,'pytorch.png']

explosionSound = ['explosion01.wav', 'explosion02.wav', 'explosion03.wav', 'explosion04.wav']

def writeScore(count):
    global gamePad
    font = pygame.font.Font('NanumGothic.ttf', 20)
    text = font.render('공부한 날:' + str(count), True, (255,255,255))
    gamePad.blit(text,(10,0))
    
def writePassed(count):
    global gamePad
    font = pygame.font.Font('NanumGothic.ttf', 20)
    text = font.render('작심삼일!:' + str(count), True, (255,0,0))
    gamePad.blit(text,(300,0))

def writeMessage(text):
    global gamePad, gameOverSound
    textfont = pygame.font.Font('NanumGothic.ttf', 80)
    text = textfont.render(text, True, (255,0,0))
    textpos = text.get_rect()
    textpos.center = (padWidth/2 , padHeight/2)
    gamePad.blit(text, textpos)
    pygame.display.update()
    pygame.mixer.music.stop()
    gameOverSound.play()
    sleep(2)
    pygame.mixer.music.play(-1)
    runGame()

# def writeImage(image):
#     global gamePad, gameClearSound
#     if 
#     gamePad.blit('dragon.png')
    
def crash():
    global gamePad
    writeMessage("국비 런!")

def gameOver():
    global gamePad
    writeMessage('취업 실패~ ㅠ')
    
# def gameClear():
#     global gamePad
#     writeImage('dragon.png')
    

def drawObject(obj, x, y):
    global gamePad
    gamePad.blit(obj,(x,y))

def initGame():
    global gamePad, clock, background, fighter, missile, explosion, missileSound, gameOverSound
    pygame.init()
    gamePad = pygame.display.set_mode((padWidth,padHeight))
    pygame.display.set_caption('국비에서 살아남기')
    background = pygame.image.load('classroom.jpg')
    fighter = pygame.image.load('fighter.png')
    missile = pygame.image.load('missile.png')
    explosion = pygame.image.load('explosion.png')
    pygame.mixer.music.load('background.mp3')
    pygame.mixer.music.play(-1)
    missileSound = pygame.mixer.Sound('missile.wav')
    gameOverSound = pygame.mixer.Sound('fail.mp3')
    # gameClearSound = pygame.mixer.Sound('clear.mp3')
    clock = pygame.time.Clock()
    
def runGame():
    global gamePad,clock,background, fighter, missile, explosion, missileSound
    
    fighterSize = fighter.get_rect().size
    fighterWidth = fighterSize[0]
    fighterHeight = fighterSize[1]
    
    x = padWidth * 0.45
    y = padHeight * 0.9
    fighterX = 0
    
    missileXY = []
    
    rock = pygame.image.load(random.choice(rockImage))
    rockSize = rock.get_rect().size
    rockWidth = rockSize[0]
    rockHeight = rockSize[1]
    destroySound = pygame.mixer.Sound(random.choice(explosionSound))
    
    rockX = random.randrange(0, padWidth - rockWidth)
    rockY = 0
    rockSpeed = 2
    
    isShot = False
    shotCount = 0
    rockPassed = 0
    
    onGame = False
    while not onGame:
        for event in pygame.event.get():
            if event.type in [pygame.QUIT]:
                pygame.quit()
                sys.exit()
            if event.type in [pygame.KEYDOWN]:
                if event.key == pygame.K_LEFT:
                    fighterX -= 5
                elif event.key == pygame.K_RIGHT:
                    fighterX += 5
                elif event.key == pygame.K_SPACE:
                    missileSound.play()
                    missileX = x + fighterWidth/2
                    missileY = y + fighterHeight
                    missileXY.append([missileX , missileY])
            
            if event.type in [pygame.K_UP]:
                if event.key == pygame.K_LEFT or event.key ==pygame.K_RIGHT:
                    fighterX = 0
        
        drawObject(background, 0, 0)
        
        x += fighterX
        if x < 0:
            x = 0
        elif x > padWidth - fighterWidth:
            x = padWidth - fighterWidth
            
        if y < rockY + rockHeight:
            if(rockX > x and rockX < x + fighterWidth) or \
                (rockX + rockWidth > x and rockX + rockWidth < x + fighterWidth):
                crash()
        
        drawObject(fighter,x,y)
        
        if rockPassed == 3:
            gameOver()
        
        if len(missileXY) != 0:
            for i, bxy in enumerate(missileXY):
                bxy[1] -= 10
                missileXY[i][1] = bxy[1]
                
                if bxy[1] < rockY:
                    if bxy[0] > rockX and bxy[0] < rockX + rockWidth:
                        missileXY.remove(bxy)
                        isShot = True
                        shotCount += 1
                
                if bxy[1] <= 0:
                    try:
                        missileXY.remove(bxy)
                    except:
                        pass
        
        if len(missileXY) != 0:
            for bx, by in missileXY:
                drawObject(missile, bx, by)
        
        writeScore(shotCount)
        
        rockY += rockSpeed
        
        if rockY > padHeight:
            rock = pygame.image.load(random.choice(rockImage))
            rockSize = rock.get_rect().size
            rockWidth = rockSize[0]
            rockHeight = rockSize[1]
            rockX = random.randrange(0, padWidth - rockWidth)
            rockY = 0
            
            rockPassed += 1
        
        writePassed(rockPassed)
            
        if isShot:
            drawObject(explosion, rockX, rockY )
            destroySound.play()
            
            rock = pygame.image.load(random.choice(rockImage))
            rockSize = rock.get_rect().size
            rockWidth = rockSize[0]
            rockHeight = rockSize[1]
            rockX = random.randrange(0, padWidth - rockWidth)
            rockY = 0
            destroySound = pygame.mixer.Sound(random.choice(explosionSound))
            isShot = False

            rockSpeed += 0.2
            if rockSpeed >= 10:
                rockSpeed = 10
            
        
        drawObject(rock , rockX, rockY)
                      
        pygame.display.update()
        
        clock.tick(60)
    
    pygame.quit()
   
initGame()
runGame()


SystemExit: 